In [5]:
# Ensure typing_extensions is upgraded and show its version
%pip install --upgrade typing_extensions
import importlib.metadata
print("typing_extensions version:", importlib.metadata.version("typing_extensions"))

Note: you may need to restart the kernel to use updated packages.
typing_extensions version: 4.15.0


In [1]:
# Install required packages
%pip install --upgrade typing_extensions

%pip install -q sentence-transformers numpy tqdm
import os
import json
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


C:\Users\sroth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Directory containing .txt files
TXT_DIR = '.'
# Output file
OUTPUT_JSON = 'embeddings.json'
# Embedding model
MODEL_NAME = 'all-MiniLM-L6-v2'

# Load model
model = SentenceTransformer(MODEL_NAME)

# Helper: read all .txt files
texts = []
for fname in os.listdir(TXT_DIR):
    if fname.endswith('.txt'):
        with open(os.path.join(TXT_DIR, fname), encoding='utf-8') as f:
            text = f.read().strip()
            if text:
                texts.append({'filename': fname, 'text': text})

# Optionally chunk large files (here: split by paragraphs > 512 chars)
CHUNK_SIZE = 512
chunks = []
for doc in texts:
    text = doc['text']
    if len(text) > CHUNK_SIZE:
        for i in range(0, len(text), CHUNK_SIZE):
            chunk = text[i:i+CHUNK_SIZE]
            chunks.append({'filename': doc['filename'], 'text': chunk})
    else:
        chunks.append({'filename': doc['filename'], 'text': text})

# Embed all chunks
embeddings = model.encode([c['text'] for c in chunks],
                          show_progress_bar=True, normalize_embeddings=True)

# Save as JSON
out = []
for i, chunk in enumerate(chunks):
    out.append({
        'filename': chunk['filename'],
        'text': chunk['text'],
        'embedding': embeddings[i].tolist()
    })

with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
    json.dump(out, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(out)} embeddings to {OUTPUT_JSON}")

Batches: 100%|██████████| 2/2 [00:00<00:00,  3.66it/s]

Wrote 33 embeddings to embeddings.json


In [3]:
# Count tokens per chunk using transformers tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
token_counts = [len(tokenizer.encode(chunk['text'])) for chunk in chunks]
print(f"Token counts per chunk: {token_counts}")
print(f"Average tokens per chunk: {sum(token_counts)/len(token_counts):.2f}")

OSError: all-MiniLM-L6-v2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`